In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tqdm import tqdm
import config

In [2]:
if config.USE_UNET: import unet as autoencoder
else: import convolutional_autoencoder as autoencoder

In [4]:
# Load the encoder model
def load_encoder(model_path, device):
    model = autoencoder.AutoEncoder()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.encoder
    model = model.to(device)
    return model

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
autoencoder_save_path = 'best_models/h1_80-10-10_Autoencoder.pth'  # Replace with your actual encoder path
encoder = load_encoder(autoencoder_save_path, device)
encoder.to(device)
encoder.eval()

In [7]:
data_transforms = {
    'bce': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'mse': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.4641, 0.4891, 0.4096], [0.1883, 0.1621, 0.2068])
    ])
}

In [8]:
full_dataset = datasets.ImageFolder(config.BASE_DIR_NOISY if config.USE_DENOISING_AUTOENCODER else config.BASE_DIR_RAW, transform=data_transforms[config.AE_TRANSFORMS])
data = DataLoader(full_dataset, batch_size=config.AE_BATCH_SIZE, shuffle=False)

In [11]:
tsne_labels = []
latent_vectors = []

with torch.no_grad():
  i = 0
  for image, label in tqdm(data):
    tsne_labels.append(label.numpy())
    image = image.to(device)
    latents = encoder(image)
    
    if config.USE_UNET:
       latents = latents[4].cpu().numpy()
    else:
       latents = latents.cpu().numpy()
	   
    latent_vectors.append(latents.reshape(latents.shape[0], -1))
    i += 1
    if i >= 1000:
      break

  8%|▊         | 999/12298 [00:33<06:18, 29.87it/s]


In [12]:
latent_vectors = np.concatenate(latent_vectors, axis=0)
labels = np.concatenate(tsne_labels, axis=0)

In [ ]:
# Apply PCA
tsne = TSNE(n_components=2, random_state=42)
tsne_result = tsne.fit_transform(latent_vectors)

# Plot the results
plt.figure(figsize=(10, 8))

scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels, cmap='viridis')

plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE of Latent Vectors')
plt.legend()
plt.show()